In [71]:
%reload_ext autoreload
%autoreload 2

In [72]:
import sys
sys.path.append('../DataRequest/')

# Amazon ML + Sinergise Blog


## Data Acquisition

This notebook shows how to obtain raw data (Sentinel 2 satellite images) using Sentinel Hub and ground truth (polygons of tulip fields) from Geopedia. At the moment Geopedia contains only data from 2016. 2017 data and other crop types (i.e. other flower types) can be easily added.

###### Required WMS instance

The instance below was created by Sinergise for the purpuse of this blog. It was created on Jed's name using his email address (jed@amazon.de). Use this instance for accessing Sentinel 2 data via Sentinel Hub.

In [73]:
WMS_INSTANCE = '71513b0b-264d-494a-b8c4-c3c36433db28'

In [74]:
sentinel_hub_wms='https://services.sentinel-hub.com/ogc/wms/'+WMS_INSTANCE

In [75]:
geopedia_layers = {'tulip_field_2016':'ttl1904', 'tulip_field_2017':'ttl1905', 'arable_land_2017':'ttl1917'}
s2_wms_layers = ['TRUE_COLOR', 'ALL_BANDS']

###### Required packages

For interactive display of Sentinel 2 images with tulip fields overlay a dev version of ipyleaflet is required, which can be found here: https://github.com/ellisonbg/ipyleaflet

To install it run:

    $ pip install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet

also install pyproj, opencv, tifffile

## Interactive display

Create map and center it around Den Helder, Netherlands. You can zoom in/out and change the view using your mouse. 

In [76]:
from ipyleaflet import Map, WMSLayer, DrawControl, Rectangle

In [77]:
zoom_level = 14

In [78]:
import math
earth_radius = 6372.7982e3
pixel_size = 2* math.pi * earth_radius * math.cos(math.radians(0))/2**(zoom_level+8)
print('Pixel dimension at zoom level %d equals %1.2f m.'%(zoom_level,pixel_size))

Pixel dimension at zoom level 14 equals 9.55 m.


In [133]:
m = Map(center=[52.9255665659715, 4.754333496093751], zoom=zoom_level, layout=dict(width='900px', height='640px'))

### Create DrawController with callback to cover a polygon with sliding patches

We will be using 256x256px images. With Sentinel-2's resolution of 10m/px, it makes sense to have our images cover 2560x2560m. This way we make full use of the satellite's capabilities. In addition, we'll have each two images overlap by 10% of their side, so stride = 2560 - 256 = 2304. 

In [134]:
# PolygonSlidingWindow is a callable class following the DrawController on_draw() API
from gather_data import PolygonSlidingWindow, GeoJsonSaver
poly_sw_cb = PolygonSlidingWindow(box_width=2560, box_height=2560, stride_x=2304, stride_y=2304)

In [81]:
# A callback of PolygonSlidingWindow to display the patches on the map as they are created
# -> Patches with green border are for training; Patches with red border are for testing
tiles = [] # save layer handles for later clean-up
def draw_patches_on_map(ipy_map):
    def aux(self, patch):
        colors = { None: '#00F', 'train': '#0F0', 'test': '#F00' }
        rect_kwargs = {'bounds': patch.box, 'weight': 1, 'color': '#00F', 'opacity': 0.2, 'fill_opacity': 0.2}
        if patch.contained:
            pg = Rectangle(fill_color=colors[patch.set], **rect_kwargs)
            ipy_map.add_layer(pg)
            tiles.append(pg)
    return aux
poly_sw_cb.register_callback(draw_patches_on_map(m))

In [135]:
# Create DrawControl with our callback on polygon creation
dc = DrawControl()
dc.on_draw(poly_sw_cb)

Add another callback to save the created polygons so they can be reused 

In [83]:
saver = GeoJsonSaver('../data/tulips/blooming/examples/17_sample_region_2/')
dc.on_draw(saver)    

In [136]:
poly_sw_cb.set_mode('train')
poly_sw_cb.load_polygons_from_folder('../data/tulips/blooming/examples/16_sample_region_2/')

DEBUG:root:Found 1 json files


In [137]:
# Check how many patches are contained in the defined polygons
patches = list(poly_sw_cb.patches.values())[0]
patches = [item for sublist in patches for item in sublist]
len(patches)

143

In [138]:
m.add_control(dc)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[52.9255665659715, 4.754333496093751], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']),), layout=Layout(height='640px', width='900px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=14)

In [36]:
poly_sw_cb.set_mode('train')

In [ ]:
poly_sw_cb.set_mode('test')

In [21]:
{ k:len([ p for v in poly for p in v if p.contained ]) for k,poly in poly_sw_cb.patches.items() }

{'train': 1562}

#### Add Sentinel 2 layer to the map

In [139]:
s2_layer = WMSLayer(url='https://services.sentinel-hub.com/v1/wms/'+WMS_INSTANCE, layers='TRUE_COLOR', tile_size=512)

In [140]:
m.add_layer(s2_layer)

#### Add ground truth layer (all tulip fields from 2017)

In [141]:
tulips = WMSLayer(url='http://service.geopedia.world/wms/ml_aws', layers=geopedia_layers['tulip_field_2017'], tile_size=512, format='image/png', version='1.3.0', TRANSPARENT=True, opacity=0.4)

In [86]:
m.add_layer(tulips)

#### openstreetmap

In [87]:
wms = WMSLayer(url='https://ows.terrestris.de/osm/service', layers='OSM-Overlay-WMS', tile_size=512)

In [88]:
m.add_layer(wms)

#### Clean-up layers

In [43]:
list(m.remove_layer(pg) for pg in tiles);

## Run downloader

In [142]:
import uuid
#root_dir = 'images/{}/'.format(uuid.uuid4())
root_dir = '../data/tulips/blooming/examples/16_sample_region_2/'

In [89]:
# A callback to display download progress on map
def show_progress_on_map(ipy_map, color):
    def aux(patch):
        if patch.contained:
            colors = { None: '#00F', 'train': '#0F0', 'test': '#F00' }
            rect_kwargs = {'bounds': patch.bounds, 'weight': 1, 'color': colors[patch.set], 'opacity': 0.2, 'fill_opacity': 0.2}
            pg = Rectangle(fill_color=color, **rect_kwargs)
            ipy_map.add_layer(pg)
            tiles.append(pg)
    return aux

In [143]:
from DataRequest import TulipFieldRequest, S2Request
# A function to download data for all the patches in the sliding window
from gather_data import BatchDownloader

In [153]:
poly_sw_cb.set_mode('train')
downloader_train_labels = BatchDownloader(root_dir, poly_sw_cb, TulipFieldRequest, (), {'width':256, 'height':256, 'crs':3857, 'layer':geopedia_layers['tulip_field_2016']})

In [ ]:
downloader_train_labels.download_data()

In [151]:
poly_sw_cb.set_mode('train')
downloader_train_images = BatchDownloader(root_dir, poly_sw_cb,
                             S2Request, (WMS_INSTANCE,),
                             {'width':256, 'height':256, 'crs':3857, 'time':('2016-04-30','2016-05-13'), 'layers':'TRUE_COLOR', 'maxcc':0.6})

In [ ]:
downloader_train_images.download_data()